In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.scrape import fetch_urls
from lib.write import write_json_lines_partition, load_jsonl_files, filter_failures
from lib.extract import extract_commander_details
import pandas as pd
import re
import json

In [3]:
commander_details_json = load_jsonl_files("../data/staging/commander_details")

In [6]:
tribed_commander_urls = [
    "https://json.edhrec.com/pages" + next(iter(commander["container"]["breadcrumb"][-1])) + tribelink["href-suffix"] + ".json"
    for commander in commander_details_json
    for tribelink in commander["panels"]["tribelinks"]["budget"] + commander["panels"]["tribelinks"]["themes"]
]

In [7]:
write_json_lines_partition(
    filter_failures(
        fetch_urls(tribed_commander_urls),
        "../data/temp/tribed_commander_fetch_errors"
    ),
    64,
    "../data/staging/tribed_commander_details"
)

Fetching URLs: 100%|██████████| 17850/17850 [2:29:46<00:00,  1.99URL/s, Est. Time Remaining=0.00s]       


In [4]:
tribed_commander_details = load_jsonl_files("../data/staging/tribed_commander_details")

In [12]:
operational_commander_details = extract_commander_details([
    tribed_commander_detail["response"]
    for tribed_commander_detail in tribed_commander_details
])
# TODO remove the response level from the staging tribed commander details files

In [14]:
json_file_directory = "../data/operational/tribed_commander_details"
write_json_lines_partition(operational_commander_details, 64, json_file_directory)